# Unificación de datos históricos

Este proceso existe para reunir, limpiar y unificar los registros de merma que hoy se encuentran dispersos en varios archivos y pestañas de Excel. A grandes rasgos, cada bloque cumple una función concreta:

Listado de archivos y parámetros básicos
Se declara la lista de libros a procesar, los índices de las columnas que interesan, la fila donde empieza el encabezado real y los nombres estándar de las columnas. Con esto se evita escribir la misma lógica una y otra vez y se garantiza que todos los archivos se lean con la misma “plantilla”.

Expresión regular sobre el nombre del archivo
Antes de abrir cada libro se extraen el año y la procedencia (bodega 321 o isla). Así cada registro queda etiquetado con metadatos coherentes sin depender de lo que haya dentro de las hojas.

Bucle de procesamiento
Para cada archivo:

Se comprueba que exista y que su nombre siga el patrón esperado.

Se cargan todas sus pestañas; dentro de cada una se seleccionan únicamente las columnas de interés, se descartan filas vacías y se añaden las columnas Year, Month y Source.

Si una hoja no tiene las columnas necesarias o devuelve un error, se salta sin interrumpir el resto del flujo.

Limpieza de campos
Las columnas numéricas (cajas, kilos, importe) se convierten a número eliminando comas y rellenando nulos con cero; las de texto se sanean para quitar caracteres mal codificados (e, ñ, tildes). Con ello se evitan futuros fallos al sumar o filtrar.

Consolidación por libro
Las hojas válidas de un mismo archivo se concatenan en un solo DataFrame. Se ordena por mes y producto para que sea fácil de inspeccionar a simple vista.

Exportación a CSV
El resultado de cada par año–origen se guarda como merma_YYYY_source.csv en UTF-8, formato ligero y multiplataforma que después puede cargarse a una base de datos, un dashboard o un notebook sin pasos extra.

El efecto final es pasar de muchos Excel heterogéneos a un conjunto de archivos CSV limpios y homogéneos. Esto simplifica análisis históricos, cálculos de KPIs y cualquier integración posterior con herramientas de business intelligence o scripts de machine learning, eliminando trabajo manual y fuentes de error.

In [ ]:
import pandas as pd
import os
import numpy as np
import re

excel_files = [
    '/content/2024 merma bod 321.xlsx',
    '/content/2024 merma isla.xlsx',
    '/content/2023 merma isla.xlsx',
    '/content/2023 merma bod 321.xlsx',
    '/content/2022 merma isla.xlsx',
    '/content/2022 merma bod 321.xlsx'
]


excel_directory = '.'

output_directory = '.'

# celdas donde estan las columnas
col_indices = [7, 10, 13, 16, 19, 21, 24]
col_names = ['Producto', 'Etiqueta', 'Tamaño', 'Color', 'Cajas', 'Kilos', 'Importe_Venta']
header_row_index = 11 # Header esta en la 12

# los nombres de las ventanas de excel
expected_months_sheets = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]


# buscamos el año y nombre de la bodega (o isla, pero pos si nos mandan otro archivo de otro lugar igual funciona)
excel_file_pattern = re.compile(r"(\d{4})\s+merma\s+(isla|bod\s*321)\.xlsx", re.IGNORECASE)

# procesamoe el excel

print("Starting processing of Excel files...")
output_files_generated = []

os.makedirs(output_directory, exist_ok=True)

for file_full_path_or_name in excel_files:
    file_path = os.path.join(excel_directory, file_full_path_or_name)

    excel_file_name = os.path.basename(file_path)

    print(f"\nProcessing file: {excel_file_name} (from path: {file_path})")

    if not os.path.exists(file_path):
        print(f"  ERROR: File not found at {file_path}. Skipping.😢")
        continue

    match = excel_file_pattern.match(excel_file_name)
    if not match:
        print(f"  WARNING: Could not parse filename '{excel_file_name}' for Year/Source. Skipping.")
        continue

    year = int(match.group(1))
    source_raw = match.group(2)
    source = 'bod' if 'bod' in source_raw.lower() else 'isla'

    all_sheets_data = []

    try:
        excel_data = pd.read_excel(file_path, sheet_name=None, header=header_row_index, engine='openpyxl')

        # Itera sobre las pestañitas
        for sheet_name, df_sheet in excel_data.items():
            month = sheet_name.strip().capitalize()
            month_upper = sheet_name.strip().upper()

            print(f"  Reading sheet: '{sheet_name}' -> Month: {month}")

            try:
                 if len(df_sheet.columns) > max(col_indices):
                     df_processed = df_sheet.iloc[:, col_indices].copy()
                     df_processed.columns = col_names
                 else:
                      print(f"  ERROR: Sheet '{sheet_name}' in {excel_file_name} has fewer columns ({len(df_sheet.columns)}) than expected index ({max(col_indices)}). Skipping sheet.")
                      continue
            except IndexError:
                 print(f"  ERROR: Could not select columns by index for sheet '{sheet_name}' in {excel_file_name}. Skipping sheet.")
                 continue
            except Exception as e_col:
                 print(f"  ERROR processing columns for sheet '{sheet_name}' in {excel_file_name}: {e_col}. Skipping sheet.")
                 continue

            df_processed.dropna(subset=['Producto'], inplace=True)

            if not df_processed.empty:
                df_processed['Year'] = year
                df_processed['Month'] = month
                df_processed['Source'] = source
                all_sheets_data.append(df_processed)
            else:
                print(f"  INFO: No data rows found in sheet '{sheet_name}' after dropping NaNs.")

        # Combina la data anterior
        if all_sheets_data:
            combined_year_source_df = pd.concat(all_sheets_data, ignore_index=True) #tirar el index
            print(f"  -> Combined {len(all_sheets_data)} sheets for {year}-{source}.")

            numeric_cols = ['Cajas', 'Kilos', 'Importe_Venta']
            for col in numeric_cols:
                if col in combined_year_source_df.columns:
                    if combined_year_source_df[col].dtype == 'object':
                       combined_year_source_df[col] = combined_year_source_df[col].astype(str).str.replace(',', '', regex=False)
                    combined_year_source_df[col] = pd.to_numeric(combined_year_source_df[col], errors='coerce')
                    combined_year_source_df[col] = combined_year_source_df[col].fillna(0)
                    if col == 'Cajas':
                        combined_year_source_df[col] = combined_year_source_df[col].astype(int)

            text_cols = ['Producto', 'Etiqueta', 'Tamaño', 'Color']
            try:
                for col in text_cols:
                    if col in combined_year_source_df.columns:
                        if combined_year_source_df[col].dtype == 'object':
                            temp_series = combined_year_source_df[col].astype(str)

                            temp_series = temp_series.str.replace('Ã±', 'ñ', regex=False).str.replace('Ã‘', 'Ñ', regex=False)
                            temp_series = temp_series.str.replace('Ã¡', 'á', regex=False).str.replace('Ã©', 'é', regex=False)

                            combined_year_source_df[col] = temp_series
            except Exception as e_enc:
                print(f"  Warning: Error during encoding correction for {excel_file_name}: {e_enc}")

            cols_order = ['Year', 'Month', 'Source'] + col_names
            final_cols_order = [col for col in cols_order if col in combined_year_source_df.columns]
            extra_cols = [col for col in combined_year_source_df.columns if col not in final_cols_order]
            combined_year_source_df = combined_year_source_df[final_cols_order + extra_cols]


            #guardarlo
            output_filename = os.path.join(output_directory, f"merma_{year}_{source}.csv")
            try:
                # Sort antes de guardar para que se vea bonito (no necesary)
                month_order_map = {
                    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
                    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
                }

                combined_year_source_df['Month_Num'] = combined_year_source_df['Month'].str.capitalize().map(month_order_map)
                combined_year_source_df_sorted = combined_year_source_df.sort_values(by=['Month_Num', 'Producto'])
                combined_year_source_df_sorted = combined_year_source_df_sorted.drop(columns=['Month_Num'])

                combined_year_source_df_sorted.to_csv(output_filename, index=False, encoding='utf-8-sig')
                print(f"  -> Saved combined data to: {output_filename} ({len(combined_year_source_df_sorted)} rows)")
                output_files_generated.append(output_filename)
            except Exception as e_save:
                print(f"  ERROR saving file {output_filename}: {e_save}")

        else:
            print(f"  -> No valid data found in any sheet for {excel_file_name}.")

    except Exception as e_excel:
        print(f"  ERROR processing Excel file {excel_file_name}: {e_excel}")


print(f"\nProcessing finished. Generated {len(output_files_generated)} CSV files:")
for fname in output_files_generated:
    print(f"- {fname}")

## Despues de que tenemos los datos unificados por año, los unificamos por bodega

In [ ]:
import pandas as pd
import os
# mejor voy a comentar en español
# btw esto es porque estoy en colab
data_dir = '/content/'


bod_files_to_combine = [
    os.path.join(data_dir, 'merma_2024_bod.csv'),
    os.path.join(data_dir, 'merma_2023_bod.csv'),
    os.path.join(data_dir, 'merma_2022_bod.csv')
]


list_of_bod_dataframes = []

# --- Proceso para archivos 'bod' ---
print("Procesando archivos 'bod'...")
# itera sobre la lista de archivos, lee cada uno y añade a la lista
for file_path in bod_files_to_combine:
    try:

        df_temp = pd.read_csv(file_path)

        list_of_bod_dataframes.append(df_temp)
        print(f"  - Archivo leído correctamente: {file_path} ({len(df_temp)} filas)")
    except FileNotFoundError:
        print(f"  - Advertencia! Archivo no encontrado: {file_path}. Se omitirá.")
    except Exception as e:
        print(f"  - Error! No se pudo leer el archivo {file_path}: {e}. Se omitirá.")


if list_of_bod_dataframes:
    # Concatena todos los df's de la lista en uno solo
    # ignore_index=True reinicia el índice del df combinado similar que arriba
    df_bod_combinado = pd.concat(list_of_bod_dataframes, ignore_index=True)

    print(f"\nDataFrame 'bod' combinado creado con éxito. Total de filas: {len(df_bod_combinado)}")

    # Define el nombre del archivo csv de salida para 'bod' (esto hay que cambiarlo para hacerlo mas general y no copiar y pegar como le hice)
    output_bod_csv_path = os.path.join(data_dir, 'merma_bod_combinado_2022-2024.csv')

    #Guardar el df combinado en un nuevo archivo csv
    # index=False evita que pandas escriba el índice del df como una columna en el csv igual que arriba
    try:
        df_bod_combinado.to_csv(output_bod_csv_path, index=False, encoding='utf-8-sig') # utf-8-sig es bueno para compatibilidad con excel las eñes sobre todo
        print(f"df 'bod' combinado guardado como: {output_bod_csv_path}")
    except Exception as e:
        print(f"error! No se pudo guardar el archivo csv combinado para 'bod': {e}")

    # Ahora tenemos el df 'df_bod_combinado'
    # las primeras 5 filas
    print("\nPrimeras 5 filas del df 'bod' combinado:")
    print(df_bod_combinado.head())

else:
    print("\nNo se pudo leer ningún archivo 'bod'. No se generó ningún df ni archivo csv combinado.")

# copié y pegué para isla

print("\n------------------------------------------")
print("Procesando archivos 'isla'...")

isla_files_to_combine = [
    os.path.join(data_dir, 'merma_2024_isla.csv'),
    os.path.join(data_dir, 'merma_2023_isla.csv'),
    os.path.join(data_dir, 'merma_2022_isla.csv')
]
list_of_isla_dataframes = []

for file_path in isla_files_to_combine:
    try:
        df_temp = pd.read_csv(file_path)
        list_of_isla_dataframes.append(df_temp)
        print(f"  - Archivo leído correctamente: {file_path} ({len(df_temp)} filas)")
    except FileNotFoundError:
        print(f"  - ¡Advertencia! Archivo no encontrado: {file_path}. Se omitirá.")
    except Exception as e:
        print(f"  - ¡Error! No se pudo leer el archivo {file_path}: {e}. Se omitirá.")

if list_of_isla_dataframes:
    df_isla_combinado = pd.concat(list_of_isla_dataframes, ignore_index=True)
    print(f"\df 'isla' combinado creado con éxito. Total de filas: {len(df_isla_combinado)}")
    output_isla_csv_path = os.path.join(data_dir, 'merma_isla_combinado_2022-2024.csv')
    try:
        df_isla_combinado.to_csv(output_isla_csv_path, index=False, encoding='utf-8-sig')
        print(f"df 'isla' combinado guardado como: {output_isla_csv_path}")
        print("\nPrimeras 5 filas del DataFrame 'isla' combinado:")
        print(df_isla_combinado.head())
    except Exception as e:
        print(f"error! No se pudo guardar el archivo csv combinado para 'isla': {e}")
else:
    print("\nNo se pudo leer ningún archivo 'isla'. No se generó ningún df ni archivo csv combinado.")

## Se genera un DF con las variables exógenas

Estos datos se han tomado de fuentes empiricas para la ciudad de Monterrey desde el 2022.

In [ ]:
import pandas as pd

fechas = pd.date_range("2022-01-01", "2024-12-01", freq="MS")

# Datos numéricos
data = {
    "temp_c": [
        13.7, 14.0, 18.0, 23.4, 27.6, 29.8, 30.3, 29.5, 25.7, 20.4, 15.8, 13.5,
        14.0, 16.2, 20.5, 25.0, 28.4, 30.1, 30.5, 30.0, 27.0, 22.5, 17.0, 14.2,
        14.5, 16.0, 20.0, 25.3, 28.8, 30.5, 30.8, 30.2, 27.5, 22.8, 17.5, 14.0
    ],
    "pres": [
        1018, 1016, 1015, 1012, 1008, 1007, 1008, 1008, 1010, 1015, 1020, 1022,
        1019, 1016, 1014, 1011, 1008, 1006, 1007, 1008, 1010, 1014, 1018, 1021,
        1019, 1016, 1014, 1011, 1008, 1006, 1007, 1008, 1010, 1014, 1018, 1021
    ],
    "gas": [
        20.9, 21.2, 21.5, 21.8, 22.0, 22.1, 22.0, 21.9, 21.7, 21.6, 21.4, 21.3,
        22.0, 22.3, 22.8, 23.6, 23.0, 23.3, 23.4, 23.3, 23.1, 23.0, 22.8, 22.6,
        23.5, 23.7, 23.9, 24.0, 24.1, 24.3, 24.5, 24.4, 24.2, 24.0, 23.8, 23.6
    ],
    "humidity": [
        54.6, 57.5, 63.3, 63.1, 67.1, 65.4, 49.7, 63.7, 71.5, 71.6, 71.0, 69.0,
        67.5, 65.7, 63.3, 63.0, 67.0, 65.0, 64.2, 63.7, 71.3, 71.6, 71.0, 66.9,
        67.5, 65.5, 63.0, 63.0, 67.0, 65.0, 64.0, 64.0, 71.0, 72.0, 71.0, 69.0
    ],
    "precip": [
        21.8, 27.3, 29.1, 33.6, 53.2, 62.6, 59.9, 72.7, 199.7, 64.1, 30.0, 22.5,
        21.8, 27.3, 29.1, 33.6, 53.2, 62.6, 59.9, 72.7, 199.7, 64.1, 30.0, 22.5,
        21.8, 27.3, 29.1, 33.6, 53.2, 62.6, 59.9, 72.7, 199.7, 64.1, 30.0, 22.5
    ],
    "cost_mwh": [
        1000, 950, 1100, 1200, 1300, 1400, 1300, 1200, 1100, 1000, 900, 950,
        1000, 980, 1100, 1150, 1250, 1300, 1200, 1150, 1100, 1000, 950, 980,
        990, 1000, 1100, 1200, 1300, 1350, 1250, 1200, 1150, 1050, 1000, 990
    ],
    "price_index": [
        120, 121, 124, 126, 129, 131, 132, 133, 135, 136, 137, 139,
        140, 142, 145, 146, 148, 150, 151, 152, 153, 154, 155, 156,
        158, 160, 163, 165, 167, 169, 170, 172, 174, 175, 176, 178
    ]
}

# Crear DataFrame con variables exógenas
df_monterrey = pd.DataFrame(data, index=fechas)

# Mostrar
df_monterrey


## Se combinan los 3 DF's así podemos tener un sólo dataframe con las variables exógenas, y los datos de Bodega e Isla

In [ ]:
# Cargar datasets de bodega e isla
df_bod = pd.read_csv("/content/merma_bod_combinado_2022-2024.csv")
df_isla = pd.read_csv("/content/merma_isla_combinado_2022-2024.csv")
df_monterrey = pd.read_csv("/content/monterrey_2022_2024.csv")

# Añadir columna de origen para trazabilidad
df_bod['Origen'] = 'Bodega'
df_isla['Origen'] = 'Isla'

# Unificar ambos datasets
df_merma = pd.concat([df_bod, df_isla], ignore_index=True)

meses = {
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
    'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
    'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

df_merma['Month'] = df_merma['Month'].astype(str).str.strip().str.lower()
df_merma['Month_Num'] = df_merma['Month'].map(meses)
df_merma['Fecha'] = pd.to_datetime(df_merma['Year'].astype(str) + '-' + df_merma['Month_Num'].astype('Int64').astype(str) + '-01', errors='coerce')

# Serie temporal combinada de mermas
df_merma['Fecha'] = pd.to_datetime(df_merma['Fecha'], utc=True)

df_monterrey_reset = df_monterrey.rename(columns={"Unnamed: 0": "Fecha"})
df_monterrey_reset['Fecha'] = pd.to_datetime(df_monterrey_reset['Fecha'], utc=True)

df = df_merma.merge(df_monterrey_reset, on="Fecha", how="left")
df.drop(labels=["Etiqueta", "Tamaño", "Color", "Importe_Venta", "Source", "Month_Num"], axis=1, inplace=True)

df.to_csv("og_exogenas.csv", index=False)